In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime
import pandas as pd
import os

In [2]:
API_KEY = 'YT_API_KEY'
channel_name = 'lekkostronniczy'
#extraction filed Channels info
CHANNELS_INFO_SNIPPED = ['title',  'publishedAt', 'country', 'description']
CHANNELS_INFO_STATS = ['viewCount', 'subscriberCount', 'videoCount']


#date format
SOURCE_DATE_FORMAT = '%Y-%m-%dT%H:%M:%SZ'
TARGET_DATE_FORMAT = '%Y-%m-%d'

Build Service

In [3]:
youtube = build('youtube', 'v3', developerKey=os.environ['YT_API_KEY'])

Get channel Id

In [4]:
request = youtube.search().list(
    part='id',
    type='channel',
    q=channel_name
)

In [5]:
try:
    response = request.execute()
except HttpError as e:
    print('Error response status code : {0}, reason : {1}'.format(e.status_code, e.error_details))

In [6]:
channelId = response['items'][0]['id']['channelId']

Get channel information

In [7]:
request = youtube.channels().list(
    part="snippet,statistics",
    id=channelId
)

try:
    response = request.execute()
except HttpError as e:
    print('Error response status code : {0}, reason : {1}'.format(e.status_code, e.error_details))

In [8]:
#E extract data for channels info table
df_columns = ['channelId'] + CHANNELS_INFO_SNIPPED
# df = pd.DataFrame(columns=df_columns)

channel_info = {}
for field in CHANNELS_INFO_SNIPPED:
    channel_info[field] = response['items'][0]['snippet'][field]
channel_info

channel_info['publishedAt']

#date conversion
date_obj = datetime.strptime(channel_info['publishedAt'], SOURCE_DATE_FORMAT)
channel_info['publishedAt'] = date_obj.strftime(TARGET_DATE_FORMAT)

#desc cleaning
channel_info['description'] = channel_info['description'].rstrip()

channel_info['channelId'] = channelId

# pd.DataFrame(channel_info)
df = pd.DataFrame(channel_info, index=[0])
df[df_columns]

,channelId,title,publishedAt,country,description
0,UC8JbbaZ_jgdsoUqrZ2bXtQQ,Lekko Stronniczy,2011-02-24,PL,Lekko Stronniczy to codzienny program rozrywko...


In [9]:

# extract data for channels statistics table
df_columns = ['channelId','extract_date'] + CHANNELS_INFO_STATS

channel_stats = {}
for field in CHANNELS_INFO_STATS:
    channel_stats[field] = response['items'][0]['statistics'][field]


channel_stats['channelId'] = channelId
channel_stats['extract_date'] = datetime.today().strftime(TARGET_DATE_FORMAT)
channel_stats
df = pd.DataFrame(channel_stats, index=[0])
df[df_columns]


,channelId,extract_date,viewCount,subscriberCount,videoCount
0,UC8JbbaZ_jgdsoUqrZ2bXtQQ,2023-04-06,201523476,399000,2082


Get channel videos informations

In [10]:
req = youtube.search().list(
    part="snippet",
    channelId=channelId,
    maxResults=10,
    order='date',
    type='video'
)

try:
    res = req.execute()
except HttpError as e:
    print('Error response status code : {0}, reason : {1}'.format(e.status_code, e.error_details))
res['items'][0].keys()

dict_keys(['kind', 'etag', 'id', 'snippet'])